
# Data Analysis Project: European Soccer Project 

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#eda">Linear Regression - Player Rating</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

European Soccer dataset contains data of matches, teams and players from soccer leagues in Europe for the time period, 2008 to 2016. I have chosen to focus first on one club, Liverpool Football Club (LFC) which plays in the English Premier League and I am also curious about the performance characteristics of all the players in the database. 

In [1]:
# import statements 
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn import linear_model
from sklearn.metrics import mean_absolute_percentage_error, r2_score


<a id='wrangling'></a>
## Data Wrangling

Connecting with SQLite database and reading the SQL tables into Pandas Dataframes.

In [2]:

# Load data from SQLite database onto Pandas dataframes

#open a connection with the SQLite database
sqll_connect = sqlite3.connect('database.sqlite')
#sqll_cursor = sqll_connect.cursor()

#read the SQL tables and load them into Pandas dataframes
match = pd.read_sql_query("SELECT * FROM Match", sqll_connect)
team = pd.read_sql_query("SELECT * FROM Team", sqll_connect)
team_att = pd.read_sql_query("SELECT * FROM Team_Attributes", sqll_connect)
#league = pd.read_sql_query("SELECT * FROM League", sqll_connect)
player = pd.read_sql_query("SELECT * FROM player", sqll_connect)
player_att = pd.read_sql_query("SELECT * FROM Player_Attributes", sqll_connect)

In [3]:
#explore the dfs
team.head(2)

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC


In [7]:
#explore the dfs
league.head(2)

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League


In [8]:
match.head(1)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.0,1.65,3.4,4.5,1.78,3.25,4.0,1.73,3.4,4.2


In [52]:
match.shape

(25979, 115)

In [4]:
#data check
#pd.set_option('display.max_rows', None)  # or 1000
match.isnull().sum()


id                0
country_id        0
league_id         0
season            0
stage             0
              ...  
GBD           11817
GBA           11817
BSH           11818
BSD           11818
BSA           11818
Length: 115, dtype: int64

In [7]:
match.duplicated().sum()

0

In [8]:
#create a new dataframe with only data from English premier league
epl_matches = match[match['league_id'] == 1729]
epl_matches.shape

(3040, 115)

In [9]:
#explore the columns of the new dataframe
epl_matches.columns

Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal',
       ...
       'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'],
      dtype='object', length=115)

In [10]:
#drop columns that will not be used in project
''' Columns dropped:
1) Columns with the position of each player;(x,y) coordinates
2) XML data on fouls, cards, crosses, corners, possession
3) Columns with betting odds from different sports betting platforms llike bet365.com
4) Country id and league id are redundant data'''

# 1) drop columns (index 11 to 54) with data on the position of each player;(x,y) coordinates
epl_matches.drop(epl_matches.columns[11:55], axis = 1, inplace = True)

C:\Users\vbatt\anaconda3\lib\site-packages\pandas\core\frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
# 2) drop data on betting odds for the game from different betting platforms
epl_matches.drop(epl_matches.loc[:,'B365H':].columns, axis = 1, inplace = True)

In [12]:
# 3) xml data provided are not relevant to the analysis and hence dropped
epl_matches.drop(epl_matches.loc[:,'goal':].columns, axis = 1, inplace = True)

In [13]:
# 4) dropping country and league columns since EPL is limited to United Kingdom
epl_matches.drop(['country_id','league_id'], axis = 1, inplace = True) 

In [14]:
epl_matches.shape

(3040, 31)

In [15]:
#checking for missing values in columns
epl_matches.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3040 entries, 1728 to 4767
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                3040 non-null   int64  
 1   season            3040 non-null   object 
 2   stage             3040 non-null   int64  
 3   date              3040 non-null   object 
 4   match_api_id      3040 non-null   int64  
 5   home_team_api_id  3040 non-null   int64  
 6   away_team_api_id  3040 non-null   int64  
 7   home_team_goal    3040 non-null   int64  
 8   away_team_goal    3040 non-null   int64  
 9   home_player_1     3040 non-null   float64
 10  home_player_2     3033 non-null   float64
 11  home_player_3     3040 non-null   float64
 12  home_player_4     3040 non-null   float64
 13  home_player_5     3037 non-null   float64
 14  home_player_6     3040 non-null   float64
 15  home_player_7     3035 non-null   float64
 16  home_player_8     3033 non-null   float

In [16]:
#create a new dataframe with only Liverpool club's data
lfc_matches = epl_matches.query('home_team_api_id == 8650 or away_team_api_id == 8650')